# TUMOR ground station data display

This code is a part of the ground station segment for [TEK5720](https://www.uio.no/studier/emner/matnat/its/TEK5720/) TUMOR (Targeting Ultraviolet Mid-Ozone layer Radiation) mission

Group Blip-Blop:
* Joachim Thomle Karlsen
* Tobias Mellum
* Michał Jan Odorczuk
* Vytenis Orlauskis
* Yawar Seraj

Code made by Michał Jan Odorczuk

This code displays the newest set of data retreived from the ground station module

Ground station module code is available [here](https://github.com/VytenisO/tumor/blob/main/flightVersion/groundStation.ino)

Ground station receives data transmitted by the [CanSat module](https://github.com/VytenisO/tumor/blob/main/flightVersion/cansat.ino)

Ground station data is saved to a file with the [data capture script](https://github.com/VytenisO/tumor/blob/main/flightVersion/data_capture.ipynb)

## imports

Please install [pyserial](https://pypi.org/project/pyserial/), [matplotlib](https://pypi.org/project/matplotlib/) and [pandas](https://pypi.org/project/pandas/)

In [ ]:
import pandas as pd
from matplotlib import pyplot as plt
from IPython.display import clear_output
from os import listdir, path
import numpy as np

## Parameters

In [ ]:
N_UV = 4
N_displayed = 100
UV_MAX = 5000

## Plotting functions

In [ ]:
def plot_uvs(axes, df, time_based, remove_extrema):
    high = 0
    for i in range(N_UV):
        uv = pd.to_numeric(df[f'UV_{i}']).to_numpy()
        al = pd.to_numeric(df[f'AL_{i}']).to_numpy()
        xs = np.arange(len(uv))
        if time_based:
            xs = pd.to_datetime(df[f'time_{i}'], format='%H:%M:%S.%f')
        axes[0, 0].plot(xs, uv, label=f'$UV_{i}$')
        axes[0, 1].plot(xs, al, label=f"$AL_{i}$")
        if remove_extrema:
            high = max(high, uv.mean() + 3 * uv.std())
        else:
            high = max(high, uv.max())

In [ ]:
def plot_magnetometer(ax, df, time_based):
    mxs, mys, mzs = [], [], []
    xss = np.array([np.arange(df.shape[0]) for i in range(N_UV)])
    for i in range(N_UV):
        mxs.append(pd.to_numeric(df[f'Mx_{i}']).to_numpy())
        mys.append(pd.to_numeric(df[f'My_{i}']).to_numpy())
        mzs.append(pd.to_numeric(df[f'Mz_{i}']).to_numpy())
        if time_based:
            xss[i,:] = pd.to_datetime(df[f'time_{i}'], format='%H:%M:%S.%f').to_numpy()
    mxs = np.array(mxs)
    mys = np.array(mys)
    mzs = np.array(mzs)
    M_N = mxs.shape[0] * mxs.shape[1]
    mxs = np.reshape(mxs.T, M_N)
    mys = np.reshape(mys.T, M_N)
    mzs = np.reshape(mzs.T, M_N)
    xss = np.reshape(xss.T, M_N)
    ax.plot(xss, mxs, label="$M_x$", color='r')
    ax.plot(xss, mys, label="$M_y$", color='g')
    ax.plot(xss, mzs, label="$M_z$", color='b')

In [ ]:
def plot_times(ax, df):
    uv_times = []
    for i in range(N_UV):
        uv_times.append(pd.to_datetime(df[f'time_{i}'], format='%H:%M:%S.%f').to_numpy()[-N_displayed:])
        ax.plot(uv_times[i], label=f'$time_{{uv,{i}}}$')
    times = pd.to_datetime(df[f'time_GPS'], format='%H:%M:%S').to_numpy()[-N_displayed:]
    ax.plot(times, label='GPS time')

In [ ]:
def plot_altitudes(ax, df, time_based, remove_extrema):
    alts = pd.to_numeric(df['alt'])
    alt_bars = pd.to_numeric(df['altitude'])
    low, high = min(alts.min(), alt_bars.min()), max(alts.max(), alt_bars.max())
    xs = np.arange(df.shape[0])
    if time_based:
        xs = pd.to_datetime(df[f'time_0'], format='%H:%M:%S.%f')
    if remove_extrema:
        low = min(alts.mean() - 3 * alts.std(), alt_bars.mean() - 3 * alt_bars.std())
        high = max(alts.mean() + 3 * alts.std(), alt_bars.mean() + 3 * alt_bars.std())
    ax.plot(xs, alts, label='GPS altitude')
    ax.plot(xs, alt_bars, label='barometric altitude')
    ax.set_ylim(low, high)

In [ ]:
def plot_temperature(ax, df, time_based, remove_extrema):
    temperatures = pd.to_numeric(df['temperature'])
    xs = np.arange(df.shape[0])
    if time_based:
        xs = pd.to_datetime(df[f'time_0'], format='%H:%M:%S.%f')
    ax.plot(xs, temperatures)
    if remove_extrema:
        ax.set_ylim(temperatures.mean() - 3 * temperatures.std(), temperatures.mean() + 3 * temperatures.std())

In [ ]:
TIME_BASED = False
REMOVE_EXTREMA = True

## Load from file and display

In [ ]:
file_times = [(f, path.getmtime(f)) for f in listdir('.') if path.isfile(f) and f[-3:] == 'csv']

sorted_files = sorted(file_times, key=lambda x: x[1], reverse=True)
data_source = sorted_files[0][0]

while True:
    df = pd.read_csv(data_source, comment='#')
    for i in range(N_UV):
        df.loc[df[f'UV_{i}']>UV_MAX,f'UV_{i}'] = UV_MAX
    # Create a figure and subplots
    fig, axes = plt.subplots(nrows=2, ncols=3, figsize=(16, 8))

    axes[0, 0].set_title('ultraviolet counts')
    axes[0, 1].set_title('ambient light counts')
    axes[0, 0].set_xticks([])
    axes[0, 0].set_ylabel('counts')
    axes[0, 1].set_xticks([])
    axes[0, 1].set_ylabel('counts')
    plot_uvs(axes, df, TIME_BASED, REMOVE_EXTREMA)

    plot_magnetometer(axes[0, 2], df, TIME_BASED)
    axes[0, 2].set_title('Magnetic field')
    axes[0, 2].set_xticks([])


    # lons = pd.to_numeric(df['lon_deg']) + pd.to_numeric(df['lon_min']) / 60 + pd.to_numeric(df['lon_sec']) / 3600
    # lats = pd.to_numeric(df['lat_deg']) + pd.to_numeric(df['lat_min']) / 60 + pd.to_numeric(df['lat_sec']) / 3600
    # axes[1, 0].plot(lons[-N_displayed:], lats[-N_displayed:])
    # axes[1, 0].scatter(lons[-1:], lats[-1:], c='r')
    # axes[1, 0].set_title('Coordinates')
    # axes[1, 0].set_xlabel('°E')
    # axes[1, 0].set_ylabel('°N')
    # plot_times(axes[1, 0], df)
    axes[1, 0].set_title('Received Signal Strength Indicator')
    rssi = pd.to_numeric(df['RSSI'])
    xs = np.arange(df.shape[0])
    if TIME_BASED:
        xs = pd.to_datetime(df[f'time_0'], format='%H:%M:%S.%f')
    axes[1, 0].plot(xs, rssi)
    axes[1, 0].set_xticks([])
    axes[1, 0].set_ylabel('dBm')

    plot_altitudes(axes[1, 1], df, TIME_BASED, REMOVE_EXTREMA)
    axes[1, 1].set_title('Altitude')
    axes[1, 1].set_xticks([])
    axes[1, 1].set_ylabel('meter above sea level')

    plot_temperature(axes[1, 2], df, TIME_BASED, REMOVE_EXTREMA)
    axes[1, 2].set_title('Temperature')
    axes[1, 2].set_xticks([])
    axes[1, 2].set_ylabel('°C')

    # Adjust layout for better appearance
    plt.tight_layout()
    clear_output(wait=True)
    # Display the subplots
    axes[0, 0].legend()
    axes[0, 1].legend()
    axes[0, 2].legend()
    # axes[1, 0].legend()
    axes[1, 1].legend()
    plt.show()


In [ ]:
file_times = [(f, path.getmtime(f)) for f in listdir('.') if path.isfile(f) and f[-3:] == 'csv']

sorted_files = sorted(file_times, key=lambda x: x[1], reverse=True)
data_source = sorted_files[0][0]

while True:
    df = pd.read_csv(data_source, comment='#')
    for i in range(N_UV):
        df.loc[df[f'UV_{i}']>UV_MAX,f'UV_{i}'] = UV_MAX
    # Create a figure and subplots
    fig, axes = plt.subplots(nrows=2, ncols=3, figsize=(16, 8))

    axes[0, 0].set_title('ultraviolet counts')
    axes[0, 1].set_title('ambient light counts')
    axes[0, 0].set_xticks([])
    axes[0, 0].set_ylabel('counts')
    axes[0, 1].set_xticks([])
    axes[0, 1].set_ylabel('counts')
    for i in range(N_UV):
        plot_uv(axes, df, i)

    plot_magnetometer(axes[0, 2], df)
    axes[0, 2].set_title('Magnetic field')
    axes[0, 2].set_xticks([])


    # lons = pd.to_numeric(df['lon_deg']) + pd.to_numeric(df['lon_min']) / 60 + pd.to_numeric(df['lon_sec']) / 3600
    # lats = pd.to_numeric(df['lat_deg']) + pd.to_numeric(df['lat_min']) / 60 + pd.to_numeric(df['lat_sec']) / 3600
    # axes[1, 0].plot(lons[-N_displayed:], lats[-N_displayed:])
    # axes[1, 0].scatter(lons[-1:], lats[-1:], c='r')
    # axes[1, 0].set_title('Coordinates')
    # axes[1, 0].set_xlabel('°E')
    # axes[1, 0].set_ylabel('°N')
    # plot_times(axes[1, 0], df)
    axes[1, 0].set_title('Received Signal Strength Indicator')
    rssi = pd.to_numeric(df['RSSI'])
    times = pd.to_datetime(df[f'time_0'], format='%H:%M:%S.%f')
    axes[1, 0].plot(times[-N_displayed:], rssi[-N_displayed:])
    axes[1, 0].set_xticks([])
    axes[1, 0].set_ylabel('dBm')

    plot_altitudes(axes[1, 1], df)
    axes[1, 1].set_title('Altitude')
    axes[1, 1].set_xticks([])
    axes[1, 1].set_ylabel('meter above sea level')

    plot_temperature(axes[1, 2], df)
    axes[1, 2].set_title('Temperature')
    axes[1, 2].set_xticks([])
    axes[1, 2].set_ylabel('°C')

    # Adjust layout for better appearance
    plt.tight_layout()
    clear_output(wait=True)
    # Display the subplots
    axes[0, 0].legend()
    axes[0, 1].legend()
    axes[0, 2].legend()
    # axes[1, 0].legend()
    axes[1, 1].legend()
    plt.show()
